In [125]:
# Import libraries

from astropy.io import ascii
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
from astropy.coordinates import EarthLocation, Angle, SkyCoord
from astropy.time import Time
from astropy.table import Table, unique
from datetime import datetime

In [126]:
# Functions

def check_date(message):
    '''
    Takes an inputted date and confirms that it is a valid future date.
    
    Parameters
    ----------
    message : str
        The message for the prompt, should accept a date in the form YYYY-MM-DD.

    Returns
    -------
    date : str
        The inputted valid date in YYYY-MM-DD format.
    '''
    # Get current date
    current_date = datetime.today()

    # Initially don't have a date
    have_date = False

    # Loop until given a valid date as input
    while have_date == False:
        # Get input and split into different parts
        date = input(message)
        try: 
            start_year = int(date[0:4])
            start_month = int(date[5:7])
            start_day = int(date[8:10])

            # Check if leap year
            if start_year % 4 == 0:
                if start_year % 400 != 0 and start_year % 100 == 0:
                    leap_year = False
                else:
                    leap_year = True
            else:
                leap_year = False

            # Check that the date is valid
            try:
                if (date[4] != '-') or (date[7] != '-') or (start_year < current_date.year) or (start_month > 12) or (start_month < current_date.month and start_year == current_date.year) or (start_day < current_date.day and start_month == current_date.month and start_year == current_date.year) or (start_month == 2 and leap_year == False and start_day > 28) or (start_month == 2 and leap_year == True and start_day > 29) or (start_day > 31) or ((start_month == 4 or start_month == 6 or start_month == 9 or start_month == 11) and start_day > 30) or (start_month <= 0) or (start_day <= 0):
                    print('Please provide a valid date in the future')
                else:
                    have_date = True
            except:
                print('Date not recognised please try again')
        
        except:
            print('Date not recognised please try again')

    return date

def check_time(message, secs=False):
    '''
    Takes an inputted timestamp and confirms that it is a valid timestamp.
    
    Parameters
    ----------
    message : str
        The message for the prompt, should accept a timestamp in the form HH:MM:SS.
    secs : bool
        Whether or not seconds are included, default False.

    Returns
    -------
    date : str
        The inputted valid timestamp in the inputted format.
    '''
    # Initially don't have a time
    have_time = False

    # Loop until given a valid time as input
    while have_time == False:
        # Get input and split into different parts
        time = input(message)
        try: 
            start_hour = int(time[0:2])
            start_minute = int(time[3:5])
            if secs == True:
                start_second = int(time[6:8])

            # Check that the date is valid
            try:
                if (time[2] != ':') or (start_hour > 23) or (start_minute > 59):
                    if secs == True and ((time[5] != ':') or start_second > 59):
                        print('Please provide a valid time')
                else:
                    have_time = True
            except:
                print('Time not recognised please try again')
                print('1')
        except:
            print('Time not recognised please try again')
            print('2')

    return time

def get_lst(loc, utc, name=None):
    '''
    Gets the LST at a point on the Earth's surface at a given time and prints it if given a name for the location, otherwise it returns the LST.

    Parameters
    ----------
    loc : EarthLocation
        The location in astropy EarthLocation format.
    utc
        The time and date in UTC that the LST is to be found at.
    name : str
        The name of the location that the LST is to be found at, default None.
    
    Returns
    -------
    lst
        The local siderial time at that location and time.
    '''
    time = Time(utc, scale='utc', location=loc)
    lst = time.sidereal_time('mean')
    if name != None: 
        print(f'The local siderial time at {name} at UTC: {utc} is: ' + str(lst))
    else:
        return lst
    
def check_file(message, columns=None):
    '''
    Checks an inputted path to a CSV and attempts to keep the specified columns, if none are specified, all columns are kept.

    Parameters
    ----------
    message : str
        The propmt for the CSV file to be dealt with.
    columns : array likee
        The list of columns to be kept in the CSV, None.
    Returns
    -------
    table
        An astropy table from the CSV.
    '''
    # Initially don't have a valid file
    have_file = False
    
    file = input(message)

    # If the file is valid we can quit, otherwise it tries again
    try:
        table = ascii.read(file)
        if columns != None:
            table.keep_columns(columns)
        have_file = True
    except:
        print('File name not recognised, please try again')
        check_file
    
    return table

In [127]:
##################################
#                                #
#   Set up times and locations   #
#                                #
##################################


valid_dates = False
valid_times = False

# Check that dates and times are in the right order
while valid_dates == False:
    start_date = check_date('Please enter starting date of observation (YYYY-MM-DD): ')
    end_date = check_date('Please enter ending date of observation (YYYY-MM-DD): ')

    if (int(start_date[0:4]) > int(end_date[0:4])) or (int(start_date[0:4]) == int(end_date[0:4]) and int(start_date[5:7]) > int(end_date[5:7])) or (start_date[0:4] == end_date[0:4] and start_date[5:7] == end_date[5:7] and int(start_date[8:10]) > int(end_date[8:10])):
        print('End date must be the same or later than the start date')
    else:
        valid_dates = True

while valid_times == False:
    start_time = check_time('Please enter starting time of observation (HH:MM:SS): ', True)
    end_time = check_time('Please enter ending time of observation (HH:MM:SS): ', True)

    if start_date == end_date:
        if (start_time == end_time) or (int(start_time[0:2]) > int(end_time[0:2])) or (int(start_time[0:2]) == int(end_time[0:2]) and int(start_time[3:5]) > int(end_time[3:5])) or (int(start_time[0:2]) == int(end_time[0:2]) and int(start_time[3:5]) == int(end_time[3:5]) and int(start_time[6:8]) >= int(end_time[6:8])):
            print('End time must be later than the start time')
        else:
            valid_times = True
    else:
        valid_times = True


starting = start_date + ' ' + start_time
ending = end_date + ' ' + end_time

starting_time = Time(starting, scale='utc')
ending_time = Time(ending, scale='utc')

# Birr
birr_loc = EarthLocation(lat=53.095*u.deg, lon=-7.922*u.deg)

# Onsala
onsala_loc = EarthLocation(lat=57.399*u.deg, lon=11.930*u.deg)


# Midpoint
mid_lon = (birr_loc.lon + onsala_loc.lon) / 2
mid_lat = (birr_loc.lat + onsala_loc.lat) / 2
mid_loc = EarthLocation(lat=mid_lat, lon=mid_lon)
LST_start_mid = get_lst(mid_loc, starting)
LST_end_mid = get_lst(mid_loc, ending)

print('Starting LST at midpoint:', LST_start_mid)
get_lst(birr_loc, starting, 'Birr')
get_lst(onsala_loc, starting, 'Onsala')


print('\n')

print('Ending LST at midpoint:', LST_end_mid)
get_lst(birr_loc, ending, 'Birr')
get_lst(onsala_loc, ending, 'Onsala')

Starting LST at midpoint: 2h51m26.06588082s
The local siderial time at Birr at UTC: 2024-07-02 08:00:00 is: 2h11m43.82588082s
The local siderial time at Onsala at UTC: 2024-07-02 08:00:00 is: 3h31m08.30588081s


Ending LST at midpoint: 9h55m31.45338122s
The local siderial time at Birr at UTC: 2024-07-03 14:59:00 is: 9h15m49.21338122s
The local siderial time at Onsala at UTC: 2024-07-03 14:59:00 is: 10h35m13.69338121s


In [128]:
# Import data and clean it up

# Exoplanet data
planets = check_file('Path to CSV file containing exoplanet hostname, coordinates and distances in NASA exoplanet database format: ', ['hostname', 'ra', 'dec', 'sy_dist'])

# One planet per host
planets = unique(planets, 'hostname')

# Sort by distance
planets.sort(keys='sy_dist')

# Get pulsar data
pulsars = check_file('Path to CSV file containing pulsar names, coordinates and luminosities in ATNF format: ', ['NAME', 'RAJ', 'DECJ', 'R_LUM'])

# Remove rows without luminosities
ind = pulsars['R_LUM'] != '*'
pulsars = pulsars[ind]

# Convert luminosities to floats
luminosities = np.array([])

for i in range(len(pulsars['R_LUM'])):
    base, exp = pulsars['R_LUM'][i].split('E')
    lum = float(base) * 10**float(exp)
    luminosities = np.append(luminosities, lum)

pulsars['R_LUM'] = luminosities

# Best to observe within 40 minutes of zenith
start_obs_psr = Angle(pulsars['RAJ'], u.hourangle).degree - 10

pulsars['start_obs'] = start_obs_psr

coords_psr = coord.SkyCoord(ra=pulsars['RAJ'], dec=pulsars['DECJ'], unit=(u.hourangle, u.deg))

In [144]:
# Optimise for closest to zenith at the middle of the observation window

# How long we'll point at a planet or pulsar
pointing_time_planet = check_time('Time to spend observing a planet (HH:MM): ')
pointing_time_planet = (int(pointing_time_planet[0:2])*u.hr + int(pointing_time_planet[3:5])*u.min + 1*u.min).to(u.hr).value

# Copy of planets that we can delete stuff from
planets_copy = Table(planets, copy=True)

time_LST = LST_start_mid.value
time_offset = 0
target_list = []
target_type = []

# The coordinates of the planets
planets_ra = np.array(planets['ra'])
planets_dec = np.array(planets['dec'])

planets_coords = SkyCoord(ra=planets_ra, dec=planets_dec, unit='deg')

# How long from the start we will observe for
obs_time = (ending_time.mjd - starting_time.mjd) * 24 # hours


# Go for obs_time hours
while time_offset <= obs_time:

    # Find the RA that is directly overhead in the middle of the observation window
    mid_lst = (time_LST + pointing_time_planet/2) * 15

    # Find target that is closest to the zenith at the middle of the observation window
    zenith = SkyCoord(ra=mid_lst, dec=mid_lat, unit='deg')
    sep = zenith.separation(planets_coords)
    ind = np.argmin(sep)
    target_list.append(planets_copy['hostname'][ind])

    # Delete that index from the coordinates array and the copy table to prevent repeats
    planets_coords = np.delete(planets_coords, ind)
    planets_copy.remove_row(ind)
    
    # Move on
    time_offset += pointing_time_planet
    time_LST += pointing_time_planet

# Output target list
print(target_list)

['TOI-3688 A', 'TOI-1696', 'TOI-3757', 'TOI-1728', 'TOI-3785', 'TOI-1199', 'TOI-2068', 'TOI-1272', 'TOI-1273', 'TOI-2048', 'TOI-2081', 'TOI-2180', 'TOI-2010', 'TOI-1431', 'TOI-1386', 'TOI-1470', 'TOI-3693', 'TOI-2120', 'TOI-1685', 'TOI-1694', 'TOI-4127', 'HD 77946', 'TOI-3807']


In [145]:
# Ger smallest value of distance times separation from zenith

# How long we'll point at a planet or pulsar
pointing_time_planet = check_time('Time to spend observing a planet (HH:MM): ')
pointing_time_planet = (int(pointing_time_planet[0:2])*u.hr + int(pointing_time_planet[3:5])*u.min + 1*u.min).to(u.hr).value

# Copy of planets that we can delete stuff from
planets_copy = Table(planets, copy=True)

time_LST = LST_start_mid.value
time_offset = 0
target_list = []
target_type = []

# The coordinates of the planets
planets_ra = np.array(planets['ra'])
planets_dec = np.array(planets['dec'])

planets_coords = SkyCoord(ra=planets_ra, dec=planets_dec, unit='deg')

# How long from the start we will observe for
obs_time = (ending_time.mjd - starting_time.mjd) * 24 # hours


# Go for obs_time hours
while time_offset <= obs_time:

    # Find the RA that is directly overhead in the middle of the observation window
    mid_lst = (time_LST + pointing_time_planet/2) * 15

    # Find target that is closest to the zenith at the middle of the observation window
    zenith = SkyCoord(ra=mid_lst, dec=mid_lat, unit='deg')
    sep = zenith.separation(planets_coords)
    sep_dist = np.array(sep * planets_copy['sy_dist'])
    ind = np.argmin(sep_dist)
    target_list.append(planets_copy['hostname'][ind])

    # Delete that index from the coordinates array and the copy table to prevent repeats
    planets_coords = np.delete(planets_coords, ind)
    planets_copy.remove_row(ind)
    
    # Move on
    time_offset += pointing_time_planet
    time_LST += pointing_time_planet

# Output target list
print(target_list)

['TOI-2120', 'TOI-1685', 'TOI-3757', 'TOI-1728', 'TOI-3785', 'TOI-1235', 'TOI-2068', 'TOI-1266', 'TOI-1411', 'TOI-2048', 'TOI-2081', 'WD 1856+534', 'HD 235088', 'TOI-1431', 'TOI-2285', 'GJ 806', 'TOI-1467', 'TOI-3688 A', 'TOI-1696', 'TOI-1634', 'TOI-1694', 'HD 77946', 'TOI-1260']


In [137]:
planets_ra = np.array(planets['ra'])
planets_dec = np.array(planets['dec'])

planets_coords = SkyCoord(ra=planets_ra, dec=planets_dec, unit='deg')

zenith = SkyCoord(ra=mid_lst, dec=mid_lat, unit='deg')

In [138]:
sep = zenith.separation(planets_coords).value

In [139]:
sep

array([80.06561142, 62.89963635, 77.67169936, 74.89117165, 67.68363963,
       49.6119095 , 55.1647263 , 56.0629912 , 29.77197968, 57.17131517,
       63.9140479 , 45.25231253, 15.33355597, 61.81873566, 70.38087715,
       52.40446674, 65.48800378, 44.03463293, 45.15466467, 41.61820491,
       56.66073593, 25.36652146, 57.27954835, 50.63763299, 12.98537366,
       62.88700682, 43.37718745, 81.2036206 , 66.96846297, 13.91483913,
        8.72918812, 27.48301714, 41.43828273, 22.3030689 , 52.87966276,
        8.64094284, 57.22654872, 60.14870356, 69.12478013, 30.64564027,
       47.12771429, 44.48763872, 47.95362871, 45.72113177, 59.07660045,
       53.63204739, 63.37334562, 22.1997979 , 53.4694549 , 83.23357842,
       79.08955285, 83.72362489, 36.11100135, 52.1257502 , 69.64681834,
       51.51503919, 69.1493435 , 19.5856192 , 80.33281713, 46.10033519,
       45.51855685, 63.38189848, 39.30052269, 50.0957182 , 26.61208945,
       76.20139613, 58.35743976, 51.70551369, 52.1233688 , 73.37

In [141]:
np.array(sep * planets['sy_dist'])

array([  964.35025677,  1411.8892673 ,  1756.9493739 ,  1730.04597796,
        1674.21574151,  1596.28799428,  1791.62894778,  1977.5435265 ,
        1072.14257771,  2127.9678049 ,  2393.05699859,  1702.17931143,
         607.73782421,  2476.79237153,  2897.62294091,  2196.22403694,
        2777.28075251,  1874.88458356,  2037.87065584,  1908.77734983,
        2943.54222978,  1342.72100736,  3070.69357957,  2785.75342261,
         789.48474782,  3920.66528521,  2815.92555296,  5907.66084311,
        4911.09874803,  1024.10015565,   696.26972365,  2323.30983343,
        3685.86066004,  1999.18018696,  5034.22850241,   859.35040687,
        5697.25200691,  6090.11638447,  7547.73474187,  3357.44441119,
        5311.76467792,  5035.15543788,  5434.82450982,  5237.30992268,
        6785.83371068,  6251.72686859,  7394.71883416,  2767.8708021 ,
        6708.4382204 , 10690.35434544, 10288.83901952, 11069.43534142,
        4967.8626773 ,  7270.70814099, 10228.19244747,  7597.2319198 ,
      